In [1]:
#@title Run to install MuJoCo and `dm_control`
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# print('Installing dm_control...')
# !pip install -q dm_control>=1.0.18

# Configure dm_control to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

print('Checking that the dm_control installation succeeded...')
try:
  from dm_control import suite
  env = suite.load('cartpole', 'swingup')
  pixels = env.physics.render()
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')
else:
  del pixels, suite

!echo Installed dm_control $(pip show dm_control | grep -Po "(?<=Version: ).+")

Sun Oct 20 15:55:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:4B:00.0 Off |                  Off |
|  0%   31C    P8             20W /  450W |     277MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title Other imports and helper functions

# General
import copy
import os
import itertools
from IPython.display import clear_output
import numpy as np

# Graphics-related
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image
# Internal loading of video libraries.

# Use svg backend for figure rendering
%config InlineBackend.figure_format = 'svg'

# Font sizes
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Inline video helper function
if os.environ.get('COLAB_NOTEBOOK_TEST', False):
  # We skip video generation during tests, as it is quite expensive.
  display_video = lambda *args, **kwargs: None
else:
  def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70

    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    # fig, ax = plt.subplots(1, 1, figsize=(scaled_width / dpi, scaled_height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Seed numpy's global RNG so that cell outputs are deterministic. We also try to
# use RandomState instances that are local to a single cell wherever possible.
np.random.seed(42)

In [3]:
import sys
import os

# Add the "env" folder to Python's module search path
env_path = "/home/zhangzhibo/CyberSpine"
if env_path not in sys.path:
    sys.path.append(env_path)

# print(sys.path)

In [4]:
import ray

ray.shutdown()
ray.init(runtime_env={"py_modules": ["/home/zhangzhibo/CyberSpine"]})

@ray.remote
def check_path():
    import sys
    return sys.path

print(ray.get(check_path.remote()))

2024-10-20 15:15:13,983	INFO worker.py:1783 -- Started a local Ray instance.
2024-10-20 15:15:14,040	INFO packaging.py:530 -- Creating a file package for local directory '/home/zhangzhibo/CyberSpine'.
2024-10-20 15:15:14,103	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_361d8c0cd4ec96a7.zip' (5.60MiB) to Ray cluster...
2024-10-20 15:15:14,216	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_361d8c0cd4ec96a7.zip'.


['/home/zhangzhibo/CyberSpine', '/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/thirdparty_files', '/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/_private/workers', '/tmp/ray/session_2024-10-20_15-15-11_472433_3165265/runtime_resources/py_modules_files/_ray_pkg_361d8c0cd4ec96a7', '/home/zhangzhibo/anaconda3/envs/mujoco/lib/python311.zip', '/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11', '/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/lib-dynload', '/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages']


In [4]:
from env import mice_env
import shimmy
from dm_control import suite

In [5]:
gym_mice_env = shimmy.DmControlCompatibilityV0(mice_env.rodent_maze_forage())

In [8]:
type(gym_mice_env.observation_space)

gymnasium.spaces.dict.Dict

In [11]:
from ray import tune
from shimmy.dm_control_compatibility import DmControlCompatibilityV0
from env import mice_env

def create_custom_env(cfg):
    return DmControlCompatibilityV0(mice_env.rodent_maze_forage())

# Register the environment with Ray
tune.register_env("MiceEnv-v0", lambda cfg: create_custom_env(cfg))


In [3]:
from ray import tune
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config


def _env_creator(ctx):
    from env import mice_env
    import gymnasium as gym
    from supersuit.generic_wrappers import resize_v1
    from ray.rllib.algorithms.dreamerv3.utils.env_runner import NormalizedImageEnv

    return NormalizedImageEnv(
        resize_v1(  # resize to 64x64 and normalize images
            gym.make(mice_env.rodent_maze_forage(), audio_on=False), x_size=64, y_size=64
        )
    )


# Register the FlappyBird-rgb-v0 env including necessary wrappers via the
# `tune.register_env()` API.
tune.register_env("CyberMice", _env_creator)


/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [5]:

# Define the `config` variable to use for training.
config = (
    DreamerV3Config()
    # set the env to the pre-registered string
    .environment("CyberMice")
    # play around with the insanely high number of hyperparameters for DreamerV3 ;) 
    .training(
        model_size="S",
        training_ratio=1024,
    )
)

# Run the tuner job.
results = tune.Tuner(trainable="DreamerV3", param_space=config).fit()

(pid=3218493) /home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=3218493)   gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(pid=3218493) /home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
(pid=3218493)   logger.warn(
(pid=3218493) /home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
(pid=3218493)   logger.warn(f"{pre} is not within the observation space.")
2024-10-20 15:54:15,088	ERROR tune_controller.py:1331 -- Trial task failed for trial DreamerV3_CyberMice_7b33c_00000
Traceback (most recent call las

(DreamerV3 pid=3218493) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::DreamerV3.__init__() (pid=3218493, ip=192.168.10.49, actor_id=22aa4b3ba25057a976a8390c01000000, repr=DreamerV3)
(DreamerV3 pid=3218493)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(DreamerV3 pid=3218493)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(DreamerV3 pid=3218493)   File "/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py", line 573, in __init__
(DreamerV3 pid=3218493)     super().__init__(
(DreamerV3 pid=3218493)   File "/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/tune/trainable/trainable.py", line 158, in __init__
(DreamerV3 pid=3218493)     self.setup(copy.deepcopy(self.config))
(DreamerV3 pid=3218493)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(DreamerV3 pid=3218493)   File "/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/rllib/algorithms/dreame

# Debug Record

24/10/15 Debug:
1. parser.parse_args() 的问题在于 Jupyter Notebook 本身不通过命令行运行，因此无法传递参数给 argparse。
2. 如果你想在 .ipynb 文件中传递参数，可以手动设置参数，而不是从命令行解析。

24/10/16 Debug:
1. RLlib 不接受用function定义的环境作为.environment的输入，
2. 必须将maze_forazing()改为gym环境或者自定义为class变量输入给env
3. 重点考虑手动定义class的方法

24/10/18 Debug:
1. 仔细看RLlib 定义custom env的方法 https://docs.ray.io/en/latest/rllib/rllib-env.html

In [4]:
import sys
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config
from ray.rllib.utils.test_utils import add_rllib_example_script_args

sys.argv = ['script_name','--num-gpus','1','--num-env-runners','4']

parser = add_rllib_example_script_args(
    default_iters=1000000,
    default_reward=800.0,
    default_timesteps=1000000
)
# Use `parser` to add your own custom command line options to this script
# and (if needed) use their values toset up `config` below.

args = parser.parse_args()


config = (
    DreamerV3Config()
    # Use image observations.
    # .environment(
    #     env="MiceEnv-v0",
    #     env_config={"from_pixels": True},
    # )
    .environment("CyberMice")
    .learners(
        num_learners=0 if args.num_gpus == 1 else args.num_gpus,
        num_gpus_per_learner=1 if args.num_gpus else 0,
    )
    .env_runners(
        num_env_runners=(args.num_env_runners or 0),
        # If we use >1 GPU and increase the batch size accordingly, we should also
        # increase the number of envs per worker.
        num_envs_per_env_runner=4 * (args.num_gpus or 1),
        remote_worker_envs=True,
    )
    .reporting(
        metrics_num_episodes_for_smoothing=(args.num_gpus or 1),
        report_images_and_videos=False,
        report_dream_data=False,
        report_individual_batch_item_stats=False,
    )
    # See Appendix A.
    .training(
        model_size="S",
        training_ratio=512,
        batch_size_B=16 * (args.num_gpus or 1),
    )

)

config.remote_worker_envs=False
rllib_algo = config.build(use_copy=False)


/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:557: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning

AssertionError: 